In [1]:
from nltk import pos_tag, word_tokenize, sent_tokenize
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import re
import random
from collections import defaultdict

In [2]:
'''
function to clean the input paragraph or paragraphs and returning sentense_wise tagged words
cleaning has to be done as '[' , ']' , '@'.. are tagged as nouns. and this program is to build blanks with nouns only
'''
def clean_and_tag(paragraph):
    # cleaning
    para = re.sub(r'[^a-zA-Z0-9 .-]','',paragraph)
    #dictionary for all sentences
    tags_dict_sentences = defaultdict(list)
    for sentence in sent_tokenize(para):
        # tagging each word in the current sentence and creating a dictionary out of em
        # actually it's enough just to care about 'NNP' and 'NN'
        temp_tags = pos_tag(word_tokenize(sentence))
        tags_dict = defaultdict(list)
        tags_dict_sentences[sentence] = tags_dict
        for i in ((temp_tags)):
            tags_dict[i[1]].append(i[0])
    return tags_dict_sentences

In [3]:
'''
function to replace a word with a blank
'''
def replaceblank(word, sentence):
    #uses regex to replace a word with a blank
    temp = re.compile(re.escape(word), re.IGNORECASE)
    return temp.sub('________________', sentence)

In [4]:
'''
function to basically remove a word and replace it with a blank if possible
As most of the geography, history, science books revolve around nouns, most fill in 
the blanks are nouns.
if you want to teach children english, you can pick adjectives or verbs as well
'''
def removeWord(sentence, tags_dict):
    #select only nouns
    words = None
    if 'NNP' in tags_dict:
        words = tags_dict['NNP']
    elif 'NN' in tags_dict:
        words = tags_dict['NN']
    else:
        print("NN and NNP not found")
        return (None, sentence, None)
    if len(words) > 0:
        # randomly choose a word, if there is a score of NNP, we can choose the 
        # best word with that
        word = random.choice(words)
        replaced = replaceblank(word, sentence)
        return (word, sentence, replaced)
    else:
        print("words are empty")
        return (None, sentence, None)

In [5]:
'''
function which combines everything and generates blanks

inputs are paragraph = paragraph(s)  you want to generate blanks from
num_of_blanks = number of blanks you want to create

right now, the function works only to create blanks upto a limit. when
number of blanks to be created are more than total sentences, we have to create 
multiple blanks from same sentence, so answers can be deduced easily. where as
if you don't care about that, we can extend the code a little and get that too

output is an array
each entry in the array is an array again, whose 1st element is the blank and
2nd element is the missing word
'''
def create_blanks(paragraph,num_of_blanks):
    # retrieve sentences whose words are tagged
    tags_dict_sentences = clean_and_tag(paragraph) # a dictionary
    # these many blanks are possible, as they can't be more than number of sentences
    possib_blanks = len(list(tags_dict_sentences.keys()))
    if possib_blanks> num_of_blanks:
        # randomly shuffle the sentences
        rand_sents = (list(tags_dict_sentences.keys()))
        random.shuffle(rand_sents)
        
        #store the blanks in an array
        blanks_arr = []
        
        # number of prepared blanks and dummy variable to iterate through rand_sents
        prepared_blanks = 0
        i = 0
        while prepared_blanks<num_of_blanks and (i<len(rand_sents)):
            curr_sent_tags = tags_dict_sentences[rand_sents[i]]
            (word, sentence, replaced) = removeWord(rand_sents[i], tags_dict_sentences[rand_sents[i]])
            if replaced is not None:
                blanks_arr.append([replaced,word])
                i+=1
                prepared_blanks+=1
            else:
                i+=1
        if prepared_blanks<num_of_blanks:
            print("sorry, couldn't form more than {} blanks".format(prepared_blanks))
        return blanks_arr
    # the same as above but asking for input
    else:
        try:
            num_of_blanks  = int(input('''number of blanks you want to create are more
            than number of sentences, please give a number less than {}.  
            if you want to quit, just press any key other than integer and enter\n'''.format(
            possib_blanks)))
            if num_of_blanks<possib_blanks:
                flag = 0
                rand_sents = (list(tags_dict_sentences.keys()))
                random.shuffle(rand_sents)
                blanks_arr = []
                prepared_blanks = 0
                i = 0
                while prepared_blanks<num_of_blanks and (i<len(rand_sents)):
                    curr_sent_tags = tags_dict_sentences[rand_sents[i]]
                    (word, sentence, replaced) = removeWord(rand_sents[i], tags_dict_sentences[rand_sents[i]])
                    if replaced is not None:
                        blanks_arr.append([replaced,word])
                        i+=1
                        prepared_blanks+=1
                    else:
                        i+=1
                if prepared_blanks<num_of_blanks:
                    print("sorry, couldn't form more than {} blanks".format(prepared_blanks))
                return blanks_arr
        except:
            print('quittin :(')
            return None
        

In [6]:
text = """A Lion lay asleep in the forest, his great head resting on his paws. A timid little Mouse came upon him unexpectedly, and in her fright and haste to
get away, ran across the Lion's nose. Roused from his nap, the Lion laid his huge paw angrily on the tiny creature to kill her.  "Spare me!" begged
the poor Mouse. "Please let me go and some day I will surely repay you."  The Lion was much amused to think that a Mouse could ever help him. But he
was generous and finally let the Mouse go.  Some days later, while stalking his prey in the forest, the Lion was caught in the toils of a hunter's
net. Unable to free himself, he filled the forest with his angry roaring. The Mouse knew the voice and quickly found the Lion struggling in the net.
Running to one of the great ropes that bound him, she gnawed it until it parted, and soon the Lion was free.  "You laughed when I said I would repay
you," said the Mouse. "Now you see that even a Mouse can help a Lion." """
file =None

In [7]:
total = 5
def createdblanks(text,total):
    output=""
    
    blanks = create_blanks(text,total)
    
    for blank in blanks:
        output = output + "<b style='color:black;'>Q )  " + blank[0] + "</b><br>"
        output = output + "<b style='color:green;'>" + "Ans: " +blank[1]+  "</b>"
        output = output + "<br>"


    # for blank in blanks:
    #     print(blank[0])
    #     print("Ans:", blank[1], "\n")
    return output


In [8]:
createdblanks(text, 5)

"<b style='color:black;'>Q )  A timid little ________________ came upon him unexpectedly and in her fright and haste toget away ran across the Lions nose.</b><br><b style='color:green;'>Ans: Mouse</b><br><b style='color:black;'>Q )  Roused from his nap the ________________ laid his huge paw angrily on the tiny creature to kill her.</b><br><b style='color:green;'>Ans: Lion</b><br><b style='color:black;'>Q )  But hewas generous and finally let the ________________ go.</b><br><b style='color:green;'>Ans: Mouse</b><br><b style='color:black;'>Q )  The ________________ was much amused to think that a Mouse could ever help him.</b><br><b style='color:green;'>Ans: Lion</b><br><b style='color:black;'>Q )  Some days later while stalking his prey in the forest the ________________ was caught in the toils of a huntersnet.</b><br><b style='color:green;'>Ans: Lion</b><br>"

In [9]:
import gradio as gr

context = gr.Textbox(lines=10, placeholder="Enter paragraph/content here...", label="Text")
total = gr.Slider(1,10, value=1,step=1, label="Total Number Of Questions")
# file = gr.File(value=r"D:\Semester 7\Final Year Project\New folder\new.txt")

output = gr.HTML( label="Question and Answers")



iface = gr.Interface(  
  fn=createdblanks, 
  inputs=[context, total], 
  outputs=output, 
  css=".gradio-container {background-image: url('file=blue.jpg')}", title="Generate Fill-in-Blanks Within Seconds",
  allow_flagging="manual",flagging_options=["Save Data"])


# iface.launch(debug=True, share=True)



In [10]:
import glob
import os.path
import pandas as pd

# folder_path = r'D:\Semester 7\Final Year Project\New folder\New folder\flagged\file'
# file_type = r'\*txt'
# files = glob.glob(folder_path + file_type)
# max_file = max(files, key=os.path.getctime)

# # import_file = pd.read_csv(max_file).replace('\n', "")
# # print(import_file)

# fp = open(max_file, 'r')
# print(fp.read())

In [11]:
total=5

In [12]:
def filecreate(file,total):
    folder_path = r'C:\Users\HP\AppData\Local\Temp'
    file_type = r'\*txt'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

# import_file = pd.read_csv(max_file).replace('\n', "")
# print(import_file)

    fp = open(max_file, 'r')
    text = fp.read()
    print(text)
    print(fp)
    generated = createdblanks(text, total)
    # return text
    return generated


In [13]:
filecreate(file,2)

A Lion lay asleep in the forest, his great head resting on his paws. A timid little Mouse came upon him unexpectedly, and in her fright and haste to
get away, ran across the Lion's nose. Roused from his nap, the Lion laid his huge paw angrily on the tiny creature to kill her.  "Spare me!" begged
the poor Mouse. "Please let me go and some day I will surely repay you."  The Lion was much amused to think that a Mouse could ever help him. But he
was generous and finally let the Mouse go.  Some days later, while stalking his prey in the forest, the Lion was caught in the toils of a hunter's
net. Unable to free himself, he filled the forest with his angry roaring. The Mouse knew the voice and quickly found the Lion struggling in the net.
Running to one of the great ropes that bound him, she gnawed it until it parted, and soon the Lion was free.  "You laughed when I said I would repay
you," said the Mouse. "Now you see that even a Mouse can help a Lion.
<_io.TextIOWrapper name='C:\\Users\\HP\

"<b style='color:black;'>Q )  A timid little Mouse came upon him unexpectedly and in her fright and haste toget away ran across the ________________ nose.</b><br><b style='color:green;'>Ans: Lions</b><br><b style='color:black;'>Q )  The ________________ knew the voice and quickly found the Lion struggling in the net.Running to one of the great ropes that bound him she gnawed it until it parted and soon the Lion was free.</b><br><b style='color:green;'>Ans: Mouse</b><br>"

In [14]:
import gradio as gr

context = gr.HTML(label="Text")
file = gr.File()
total = gr.Slider(1,10, value=1,step=1, label="Total Number Of Questions")


# output = gr.HTML( label="Question and Answers")



fface = gr.Interface(  
  fn=filecreate, 
  inputs=[file,total], 
  outputs=context, 
  css=".gradio-container {background-image: url('file=blue.jpg')}", title="Generate Fill-in-Blanks Within Seconds",
  allow_flagging="manual",flagging_options=["Save Data"])


# fface.launch(debug=True, share=True)



In [15]:
demo = gr.TabbedInterface([iface, fface], ["Text", "Upload File"])
demo.launch(debug=True, share=True)


c:\Users\HP\anaconda3\lib\site-packages\gradio\blocks.py:726: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

Could not create share link, please check your internet connection.


quittin :(


Traceback (most recent call last):
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\routes.py", line 321, in run_predict
    output = await app.blocks.process_api(
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\blocks.py", line 1015, in process_api
    result = await self.call_function(fn_index, inputs, iterator, request)
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\blocks.py", line 856, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_4580\18841

quittin :(


Traceback (most recent call last):
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\routes.py", line 321, in run_predict
    output = await app.blocks.process_api(
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\blocks.py", line 1015, in process_api
    result = await self.call_function(fn_index, inputs, iterator, request)
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\blocks.py", line 856, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_4580\18841

In [ ]:
# import gradio as gr
# import csv, io, tempfile


# def getfile(file):


#   filename = tempfile.NamedTemporaryFile()
#   print(filename)
  
#   fp = filename.read()
#   text = fp.read()
#   print(text)
#   return text
#   # files = glob.glob(file)
#   # max_file = max(files, key=os.path.getctime)  
#   # return max_file

# context = gr.Textbox(label="Text")
# file = gr.File()


# # output = gr.HTML( label="Question and Answers")



# gface = gr.Interface(  
#   fn=getfile, 
#   inputs=[file], 
#   outputs=context,
#   css=".gradio-container {background-image: url('file=blue.jpg')}", title="Generate Fill-in-Blanks Within Seconds",
#   allow_flagging="manual",flagging_options=["Save Data"])


# gface.launch(debug=True, share=True)



Running on local URL:  http://127.0.0.1:7866

Could not create share link, please check your internet connection.


Traceback (most recent call last):
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\routes.py", line 321, in run_predict
    output = await app.blocks.process_api(
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\blocks.py", line 1015, in process_api
    result = await self.call_function(fn_index, inputs, iterator, request)
  File "c:\Users\HP\anaconda3\lib\site-packages\gradio\blocks.py", line 856, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "c:\Users\HP\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_13532\3887

Keyboard interruption in main thread... closing server.
